In [ ]:
import pandas
import os

# This query represents dataset "all wgs surveys" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_73282597_survey_sql = """
    SELECT DISTINCT
        answer.person_id,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer  
 """

dataset_73282597_survey_df = pandas.read_gbq(
    dataset_73282597_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_73282597_survey_df.head(5)

In [ ]:
bucket=os.getenv('WORKSPACE_BUCKET')


In [ ]:
import pandas as pd
import os

In [ ]:
# This query represents dataset "ced2" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_51517751_person_sql = """
    SELECT
        person.person_id 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person   
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (836793) 
                                AND is_standard = 0  
                                AND  value_source_concept_id IN (1384519)
                            )) criteria 
                    UNION
                    ALL SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (
                                    SELECT
                                        DISTINCT c.concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                    JOIN
                                        (
                                            select
                                                cast(cr.id as string) as id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                            WHERE
                                                concept_id IN (194992) 
                                                AND full_text LIKE '%_rank1]%'
                                        ) a 
                                            ON (
                                                c.path LIKE CONCAT('%.',
                                            a.id,
                                            '.%') 
                                            OR c.path LIKE CONCAT('%.',
                                            a.id) 
                                            OR c.path LIKE CONCAT(a.id,
                                            '.%') 
                                            OR c.path = a.id) 
                                        WHERE
                                            is_standard = 1 
                                            AND is_selectable = 1
                                        ) 
                                        AND is_standard = 1 
                                )
                            ) criteria 
                        ) )"""

dataset_51517751_person_df = pandas.read_gbq(
    dataset_51517751_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_51517751_person_df.head(5)

In [ ]:
#label patients
dataset_73282597_survey_df['CeD']='healthy'
dataset_73282597_survey_df.loc[dataset_73282597_survey_df['person_id'].isin(dataset_51517751_person_df.person_id),'CeD']='celiac'

In [ ]:
#we will not analyze the covid-19 surveys
survey_df = dataset_73282597_survey_df[~dataset_73282597_survey_df['survey'].str.contains('COVID')]

In [ ]:
#count answers by CeD status
survey= survey_df.value_counts(['survey','question_concept_id','question','answer','CeD'])

surv_count = survey.reset_index().pivot(columns='CeD',index=['survey','question_concept_id','question','answer'],values='count').fillna(0).reset_index()

In [ ]:
#count question and survey responses per condition
question_cond_cts = survey_df[['person_id','question','CeD']].drop_duplicates().value_counts(['question','CeD'])
survey_cond_cts = survey_df[['person_id','survey','CeD']].drop_duplicates().value_counts(['survey','CeD'])

question_cond = pandas.DataFrame(question_cond_cts).reset_index().pivot(index='question',columns='CeD',values='count').fillna(0)
survey_cond = pandas.DataFrame(survey_cond_cts).reset_index().pivot(index='survey',columns='CeD',values='count').fillna(0)

question_cond.rename(columns={'celiac':'CeD question respondants','healthy':'healthy question respondants'},inplace=True)
survey_cond.rename(columns={'celiac':'CeD survey respondants','healthy':'healthy survey respondants'},inplace=True)

In [ ]:
surveys2 = pandas.merge(surv_count,survey_cond, on='survey',how='left')
surv3 = pandas.merge(surveys2,question_cond, on='question',how='left')
surv3

In [ ]:
#uncomment to check if the counts make sense
#surv_check=surv3.groupby('question').agg({'celiac':sum,'healthy':sum,'CeD survey respondants':'min', 'healthy survey respondants':'min',
 #      'CeD question respondants':'min', 'healthy question respondants':'min'})

#surv_check[surv_check['healthy']<surv_check['healthy question respondants']]
#looks good

In [ ]:
surv3.to_csv('survey_cats_count.csv',index=False)

In [ ]:
surv3=pd.read_csv('survey_cats_count.csv')

In [ ]:
dupid.drop_duplicates(['person_id','question'])
#no person has duplicate answers for any question

## Chi-square and odds ratio

In [ ]:
from scipy.stats import chisquare, fisher_exact

import statsmodels.stats.multitest as st

import numpy as np

In [ ]:
#gather sum by question as pseudo-total
sums = surv3.groupby('question').agg({'celiac':sum,'healthy':sum})
sums.columns=['celtot chi-sq','healthytot chi-sq']
surv5 = pd.merge(surv3,sums, on='question',how='left')
#define expected counts based on distribution of healthy respondants
surv5['exp freq']=surv5['healthy']/surv5['healthytot chi-sq']
surv5['exp cel']=surv5['exp freq']*surv5['celtot chi-sq']

In [ ]:
#chi-square by question
for i in surv5['question'].unique():
  print(i)
  exp = surv5.loc[surv5['question']==i,['exp cel']]
  obs = surv5.loc[surv5['question']==i,['celiac']]
  res = chisquare(obs,exp)
  surv5.loc[surv5['question']==i,['pval']]=res.pvalue[0]
  surv5.loc[surv5['question']==i,['chi-sq']]=res.statistic[0]

In [ ]:
surv5['OR']=surv5['celiac']*(surv5['healthy question respondants']-surv5['healthy'])/(surv5['healthy']*(surv5['CeD question respondants']-surv5['celiac']))

In [ ]:
#need only 1 p-value per question; drop duplicates in these 2 to keep the FDR from being too high
fdr_df=surv5[['question','pval']].drop_duplicates()
#FDR correction as defined previously
def FDR(sample, pval):
    #first sort by pval
    sample.sort_values(by=pval, inplace = True)
    samp = sample.dropna(subset=[pval])
    pvals = samp[pval]
    #calculation of correction
    padj = st.fdrcorrection(pvals, is_sorted=True)
    padded_col = np.pad(padj[1], (0, len(sample) - len(padj[1])), 'constant', constant_values=(np.nan, np.nan))
    sample['Padj'] = padded_col
#execute
FDR(fdr_df,'pval')

In [ ]:
surv5_corr = pd.merge(surv5,fdr_df,on=['question','pval'],how='left')

In [ ]:
surv5_corr.to_csv('survey_cats_fdr.csv',index=False)